In [1]:
import torch
import os
import numpy as np
import random 

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

SEED = 31
seed_everything(SEED)

import flair
flair.device = torch.device('cuda:1') 

In [2]:
from oracle import HybridOracle
from active_learner import ActiveLearner

In [3]:
EXPERIMENT = 'weighted_sampling_small_bs'

In [4]:
oracle = HybridOracle(
    experiment_name=EXPERIMENT,
    all_data_file='../twitter_dataset/train_10.csv',
    valid_file='../twitter_dataset/valid.csv',
    test_file='../twitter_dataset/test.csv'
)

../twitter_dataset/train_10.csv
Starting with labelled data


In [5]:
sentences = oracle.get_all_sentences()

In [6]:
learner = ActiveLearner(sentences,
                        experiment_name=EXPERIMENT,
                        oracle=oracle, 
                        embeddings_storage_mode='gpu')

In [7]:
from flair.embeddings import WordEmbeddings, FlairEmbeddings, DocumentRNNEmbeddings
from flair.models import TextClassifier
from flair.data import Dictionary, Sentence

In [8]:
word_embeddings = [WordEmbeddings('glove'),

                   # comment in flair embeddings for state-of-the-art results
                    FlairEmbeddings('news-forward'),
                    FlairEmbeddings('news-backward'),
                   ]

# 4. initialize document embedding by passing list of word embeddings
# Can choose between many RNN types (GRU by default, to change use rnn_type parameter)
document_embeddings: DocumentRNNEmbeddings = DocumentRNNEmbeddings(word_embeddings,
                                                                     hidden_size=512,
                                                                     reproject_words=True,
                                                                     reproject_words_dimension=256,
                                                                     )

label_dict = Dictionary()
label_dict.idx2item = [b'0', b'1']
label_dict.item2idx = {b'0': 0, b'1': 1}
# 5. create the text classifier
classifier = TextClassifier(document_embeddings, label_dictionary=label_dict)

In [9]:
classifier = classifier.load('../twitter_dataset/0_1/best-model.pt')

2019-10-19 13:35:29,958 loading file ../twitter_dataset/0_1/best-model.pt


In [10]:
classifier, opt_state = learner.step(classifier, step_num=0, max_sample_size=100000, labelling_step_size=32, step_lr=1e-5)

STARTING LABELLING
DONE 1/32 LABELS
DONE 2/32 LABELS
DONE 3/32 LABELS
DONE 4/32 LABELS
DONE 5/32 LABELS
DONE 6/32 LABELS
DONE 7/32 LABELS
DONE 8/32 LABELS
DONE 9/32 LABELS
DONE 10/32 LABELS
DONE 11/32 LABELS
DONE 12/32 LABELS
DONE 13/32 LABELS
DONE 14/32 LABELS
DONE 15/32 LABELS
DONE 16/32 LABELS
DONE 17/32 LABELS
DONE 18/32 LABELS
DONE 19/32 LABELS
DONE 20/32 LABELS
DONE 21/32 LABELS
DONE 22/32 LABELS
DONE 23/32 LABELS
DONE 24/32 LABELS
DONE 25/32 LABELS
DONE 26/32 LABELS
DONE 27/32 LABELS
DONE 28/32 LABELS
DONE 29/32 LABELS
DONE 30/32 LABELS
DONE 31/32 LABELS
DONE 32/32 LABELS
2019-10-19 13:35:50,848 Reading data from weighted_sampling_small_bs
2019-10-19 13:35:50,849 Train: weighted_sampling_small_bs/labelled_32_1571492150.csv
2019-10-19 13:35:50,850 Dev: weighted_sampling_small_bs/valid.csv
2019-10-19 13:35:50,850 Test: weighted_sampling_small_bs/test.csv
2019-10-19 13:35:50,927 ----------------------------------------------------------------------------------------------------
201

/home/kunwar31/anaconda3/envs/torch/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type DocumentRNNEmbeddings. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/kunwar31/anaconda3/envs/torch/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type StackedEmbeddings. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/kunwar31/anaconda3/envs/torch/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type WordEmbeddings. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/kunwar31/anaconda3/envs/torch/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type FlairEmb

2019-10-19 13:37:12,256 ----------------------------------------------------------------------------------------------------
2019-10-19 13:37:12,258 Testing using best model ...
2019-10-19 13:37:12,259 loading file weighted_sampling_small_bs/0/best-model.pt
2019-10-19 13:38:31,268 0.7036	0.7037	0.7036
2019-10-19 13:38:31,269 
MICRO_AVG: acc 0.5428 - f1-score 0.7036
MACRO_AVG: acc 0.5428 - f1-score 0.7036
0          tp: 3506 - fp: 1479 - fn: 1484 - tn: 3532 - precision: 0.7033 - recall: 0.7026 - accuracy: 0.5420 - f1-score: 0.7029
1          tp: 3531 - fp: 1485 - fn: 1479 - tn: 3506 - precision: 0.7039 - recall: 0.7048 - accuracy: 0.5436 - f1-score: 0.7043
2019-10-19 13:38:31,270 ----------------------------------------------------------------------------------------------------


In [ ]:
for step_num in range(1, 400):
    classifier, opt_state = learner.step(classifier, step_num=step_num, optimizer_state=opt_state, max_sample_size=100000, labelling_step_size=32, step_lr=1e-5)

STARTING LABELLING
DONE 1/32 LABELS
DONE 2/32 LABELS
DONE 3/32 LABELS
DONE 4/32 LABELS
DONE 5/32 LABELS
DONE 6/32 LABELS
DONE 7/32 LABELS
DONE 8/32 LABELS
DONE 9/32 LABELS
DONE 10/32 LABELS
DONE 11/32 LABELS
DONE 12/32 LABELS
DONE 13/32 LABELS
DONE 14/32 LABELS
DONE 15/32 LABELS
DONE 16/32 LABELS
DONE 17/32 LABELS
DONE 18/32 LABELS
DONE 19/32 LABELS
DONE 20/32 LABELS
DONE 21/32 LABELS
DONE 22/32 LABELS
DONE 23/32 LABELS
DONE 24/32 LABELS
DONE 25/32 LABELS
DONE 26/32 LABELS
DONE 27/32 LABELS
DONE 28/32 LABELS
DONE 29/32 LABELS
DONE 30/32 LABELS
DONE 31/32 LABELS
DONE 32/32 LABELS
2019-10-19 13:38:47,038 Reading data from weighted_sampling_small_bs
2019-10-19 13:38:47,039 Train: weighted_sampling_small_bs/labelled_32_1571492327.csv
2019-10-19 13:38:47,042 Dev: weighted_sampling_small_bs/valid.csv
2019-10-19 13:38:47,043 Test: weighted_sampling_small_bs/test.csv
2019-10-19 13:38:47,094 ----------------------------------------------------------------------------------------------------
201

/home/kunwar31/anaconda3/envs/torch/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type DocumentRNNEmbeddings. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/kunwar31/anaconda3/envs/torch/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type StackedEmbeddings. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/kunwar31/anaconda3/envs/torch/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type WordEmbeddings. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/kunwar31/anaconda3/envs/torch/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type FlairEmb

2019-10-19 13:40:07,905 ----------------------------------------------------------------------------------------------------
2019-10-19 13:40:07,907 Testing using best model ...
2019-10-19 13:40:07,908 loading file weighted_sampling_small_bs/1/best-model.pt
2019-10-19 13:41:25,913 0.703	0.7031	0.703
2019-10-19 13:41:25,916 
MICRO_AVG: acc 0.5421 - f1-score 0.703
MACRO_AVG: acc 0.5421 - f1-score 0.70305
0          tp: 3563 - fp: 1542 - fn: 1427 - tn: 3469 - precision: 0.6979 - recall: 0.7140 - accuracy: 0.5455 - f1-score: 0.7059
1          tp: 3468 - fp: 1428 - fn: 1542 - tn: 3563 - precision: 0.7083 - recall: 0.6922 - accuracy: 0.5387 - f1-score: 0.7002
2019-10-19 13:41:25,916 ----------------------------------------------------------------------------------------------------
STARTING LABELLING
DONE 1/32 LABELS
DONE 2/32 LABELS
DONE 3/32 LABELS
DONE 4/32 LABELS
DONE 5/32 LABELS
DONE 6/32 LABELS
DONE 7/32 LABELS
DONE 8/32 LABELS
DONE 9/32 LABELS
DONE 10/32 LABELS
DONE 11/32 LABELS
DONE 

/home/kunwar31/anaconda3/envs/torch/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type DocumentRNNEmbeddings. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/kunwar31/anaconda3/envs/torch/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type StackedEmbeddings. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/kunwar31/anaconda3/envs/torch/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type WordEmbeddings. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/kunwar31/anaconda3/envs/torch/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type FlairEmb

2019-10-19 13:43:15,470 ----------------------------------------------------------------------------------------------------
2019-10-19 13:43:15,472 Testing using best model ...
2019-10-19 13:43:15,474 loading file weighted_sampling_small_bs/2/best-model.pt
2019-10-19 13:44:27,129 0.7025	0.7026	0.7025
2019-10-19 13:44:27,131 
MICRO_AVG: acc 0.5415 - f1-score 0.7025
MACRO_AVG: acc 0.5414 - f1-score 0.7024
0          tp: 3616 - fp: 1600 - fn: 1374 - tn: 3411 - precision: 0.6933 - recall: 0.7246 - accuracy: 0.5487 - f1-score: 0.7086
1          tp: 3410 - fp: 1375 - fn: 1600 - tn: 3616 - precision: 0.7126 - recall: 0.6806 - accuracy: 0.5341 - f1-score: 0.6962
2019-10-19 13:44:27,132 ----------------------------------------------------------------------------------------------------
STARTING LABELLING
DONE 1/32 LABELS
DONE 2/32 LABELS
DONE 3/32 LABELS
DONE 4/32 LABELS
DONE 5/32 LABELS
DONE 6/32 LABELS
DONE 7/32 LABELS
DONE 8/32 LABELS
DONE 9/32 LABELS
DONE 10/32 LABELS
DONE 11/32 LABELS
DON

...

/home/kunwar31/anaconda3/envs/torch/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type DocumentRNNEmbeddings. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/kunwar31/anaconda3/envs/torch/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type StackedEmbeddings. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/kunwar31/anaconda3/envs/torch/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type WordEmbeddings. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/kunwar31/anaconda3/envs/torch/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type FlairEmb

2019-10-19 14:44:45,880 ----------------------------------------------------------------------------------------------------
2019-10-19 14:44:45,881 Testing using best model ...
2019-10-19 14:44:45,884 loading file weighted_sampling_small_bs/21/best-model.pt
2019-10-19 14:45:55,999 0.6529	0.653	0.6529
2019-10-19 14:45:56,001 
MICRO_AVG: acc 0.4847 - f1-score 0.6529
MACRO_AVG: acc 0.4669 - f1-score 0.6305499999999999
0          tp: 4496 - fp: 2976 - fn: 494 - tn: 2035 - precision: 0.6017 - recall: 0.9010 - accuracy: 0.5644 - f1-score: 0.7215
1          tp: 2034 - fp: 495 - fn: 2976 - tn: 4496 - precision: 0.8043 - recall: 0.4060 - accuracy: 0.3695 - f1-score: 0.5396
2019-10-19 14:45:56,002 ----------------------------------------------------------------------------------------------------
STARTING LABELLING
DONE 1/32 LABELS
DONE 2/32 LABELS
DONE 3/32 LABELS
DONE 4/32 LABELS
DONE 5/32 LABELS
DONE 6/32 LABELS
DONE 7/32 LABELS
DONE 8/32 LABELS
DONE 9/32 LABELS
DONE 10/32 LABELS
DONE 11/32 

/home/kunwar31/anaconda3/envs/torch/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type DocumentRNNEmbeddings. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/kunwar31/anaconda3/envs/torch/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type StackedEmbeddings. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/kunwar31/anaconda3/envs/torch/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type WordEmbeddings. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/kunwar31/anaconda3/envs/torch/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type FlairEmb

2019-10-19 14:47:35,979 ----------------------------------------------------------------------------------------------------
2019-10-19 14:47:35,981 Testing using best model ...
2019-10-19 14:47:35,982 loading file weighted_sampling_small_bs/22/best-model.pt
2019-10-19 14:48:45,478 0.6491	0.6492	0.6491
2019-10-19 14:48:45,480 
MICRO_AVG: acc 0.4806 - f1-score 0.6491
MACRO_AVG: acc 0.4611 - f1-score 0.6244000000000001
0          tp: 4530 - fp: 3048 - fn: 460 - tn: 1963 - precision: 0.5978 - recall: 0.9078 - accuracy: 0.5636 - f1-score: 0.7209
1          tp: 1962 - fp: 461 - fn: 3048 - tn: 4530 - precision: 0.8097 - recall: 0.3916 - accuracy: 0.3586 - f1-score: 0.5279
2019-10-19 14:48:45,481 ----------------------------------------------------------------------------------------------------
STARTING LABELLING
DONE 1/32 LABELS
DONE 2/32 LABELS
DONE 3/32 LABELS
DONE 4/32 LABELS
DONE 5/32 LABELS
DONE 6/32 LABELS
DONE 7/32 LABELS
DONE 8/32 LABELS
DONE 9/32 LABELS
DONE 10/32 LABELS
DONE 11/32

/home/kunwar31/anaconda3/envs/torch/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type DocumentRNNEmbeddings. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/kunwar31/anaconda3/envs/torch/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type StackedEmbeddings. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/kunwar31/anaconda3/envs/torch/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type WordEmbeddings. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
